In [0]:
import pandas as pd
import numpy as np
import numpy as np
import sqlite3

In [0]:

file_path = "/Workspace/Users/joaovab@al.insper.edu.br/Hackathon_Terra_Signal_grupo_1/data/history.csv"
df = pd.read_csv(file_path)
df.info()

In [0]:
filtro_vazios = df[df['TotalCharges'].isna()]
df['tenure'] = pd.to_numeric(df['tenure'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(df['tenure'] * df['MonthlyCharges'])
df.dropna(subset=['TotalCharges'], inplace=True)

In [0]:
colunas = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'MultipleLines']
for col in colunas:
    df[col] = df[col].replace({'No internet service': 'No', 'No phone service': 'No'})

df['PhoneService'] = df['PhoneService'].str.capitalize()
df['Churn'] = df['Churn'].astype(str).str.strip()
df['Churn'] = np.where(df['Churn'].str.lower() == 'yes', 1, 0)
df.to_csv('../data/history_clean.csv', index=False)

In [0]:
spark_df = spark.createDataFrame(df)
nome_tabela_uc = "workspace.churn_zero.history_clean"
spark_df.write.format("delta").mode("overwrite").saveAsTable(nome_tabela_uc)

In [0]:
%sql
CREATE OR REPLACE TABLE workspace.churn_zero.history_silver AS
SELECT 
    *,
    
    -- 1. Sentimento
    ai_analyze_sentiment(CustomerFeedback) as sentiment_label,
    
    -- 2. Tópico (Labels ajustados para precisão)
   ai_gen(
            'Atue como um Analista de Retenção Sênior. Classifique o feedback em UMA categoria: 
        [sales_opportunity, lost_sales_opportunity, high_price, internet_connection, billing_issues, competitor_offer, contract_terms, personal_reasons, customer_service, sales_misunderstanding, general_satisfaction, service_quality, other].

        PERFIL:
        - Feedback: "' || CustomerFeedback || '"
        - Tem Internet: ' || InternetService || '
        
        REGRAS OBRIGATÓRIAS:
        1. OPORTUNIDADE (Priority Check): 
           - Se "Tem Internet" for "No" E o texto disser "wish that internet was included", "wish they offered internet", classifique como "sales_opportunity".
           - POREM, se o texto deixar claro que o cliente JÁ TOMOU A DECISÃO DE SAIR por causa disso ("ultimately decided to switch", "cancelled service due to lack of internet"), classifique como "lost_sales_opportunity".
        
        2. TÉCNICO: Se falar de "slow", "speed", "outage", "reliability", USE "internet_connection". NÃO use "service_quality" genericamente.
        3. VENDA ERRADA: Se "Tem Internet" for "No" mas ele reclamar que a conexão dele (da nossa empresa) está lenta, use "sales_misunderstanding".
        4. PREÇO: "Caro" = high_price. "Concorrente melhor" = competitor_offer.
        
        SAÍDA: Apenas o nome da categoria.'
        ) as feedback_topic,

    -- 3. Score Numérico
    CASE 
        WHEN ai_analyze_sentiment(CustomerFeedback) = 'positive' THEN 1 
        ELSE 0 
    END as sentiment_score

FROM workspace.churn_zero.history_clean;

In [0]:
df = spark.sql("SELECT * FROM workspace.churn_zero.history_silver")
df.toPandas().to_csv("/Workspace/Users/joaovab@al.insper.edu.br/Hackathon_Terra_Signal_grupo_1/data/history_silver.csv")

In [0]:
%sql
SELECT 
    customerID,
    Churn,
    feedback_topic,
    MonthlyCharges,
    InternetService,
    
    ai_gen(
        'Atue como um Estrategista de Vendas Sênior da Terra Signal. 
        Sua tarefa é gerar uma **Instrução Interna Curta** para o vendedor que vai atender este cliente.
        Não escreva um diálogo. Escreva uma diretriz: "O cliente tem X, ofereça Y".

        DADOS DO CLIENTE:
        - Status Atual: ' || CASE WHEN Churn = 1 THEN 'CANCELOU/RISCO CRÍTICO' ELSE 'ATIVO' END || '
        - Motivo/Feedback: ' || feedback_topic || '
        - Tecnologia: ' || InternetService || '
        - Valor Mensal Pago: $' || MonthlyCharges || '
        - Contrato: ' || Contract || '
        - Qtd Add-ons: ' || 
          (CASE WHEN OnlineSecurity != 'No' THEN 1 ELSE 0 END + 
           CASE WHEN OnlineBackup != 'No' THEN 1 ELSE 0 END + 
           CASE WHEN DeviceProtection != 'No' THEN 1 ELSE 0 END + 
           CASE WHEN TechSupport != 'No' THEN 1 ELSE 0 END +
           CASE WHEN StreamingTV != 'No' THEN 1 ELSE 0 END +
           CASE WHEN StreamingMovies != 'No' THEN 1 ELSE 0 END) || '

        RACIOCÍNIO ESTRATÉGICO (USE ISSO PARA DECIDIR):
        1. SE "CANCELOU/RISCO CRÍTICO": O foco é resolver a dor exata.
           - Se reclamou de Preço ("high_price") e paga muito (>$80), autorize desconto de 15% em troca de fidelidade.
           - Se reclamou de Técnico ("internet_connection"), ofereça visita técnica VIP ou upgrade de modem.
           - Se foi "lost_sales_opportunity", ofereça exatamente o que ele queria e não tinha.
        
        2. SE "ATIVO": O foco é aumentar valor (Upsell) ou blindar (Lock-in).
           - Se não tem internet ("No") e é sales_opportunity: Venda agressiva do pacote Fibra.
           - Se tem poucos Add-ons (0 ou 1): Ofereça um serviço barato (ex: TechSupport) por 3 meses grátis para gerar hábito.
           - Se paga pouco (<$30) e está feliz: Tente subir o plano.
           - Se paga muito (>$90): Apenas agradeça e reforce os benefícios para não despertar vontade de cancelar.

        FORMATO DA SAÍDA:
        "Contexto: [Resumo em poucas palavras]. Ação: [Ordem direta do que oferecer]."
        '
    ) as acao_recomendada_vendedor

FROM workspace.churn_zero.history_silver
LIMIT 250;